In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo", ]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

This module was written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`) and other things in Blender from Jupyter without opening it.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import bpy
import math
import os
from PIL import Image
import pyexiv2

In [ ]:
def open_blend(**kwargs):
    """Open a Blender file to be used with scripts."""
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))
    name = kwargs.pop('name', 'phstudio_ArtemisStatue.blend')
    path = kwargs.pop('path',
                      os.path.join(s3path, 'blend-meshes', name))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    try:
        bpy.ops.wm.open_mainfile(filepath=path)
        print('Opened:', path)
    except IOError:
        print('No such file:', path)

In [ ]:
def set_up_sc(**kwargs):
    """Set up a scene ('bpy.context.scene') and return it."""
    wpx = kwargs.pop('wpx', 320)
    hpx = kwargs.pop('hpx', 240)
    antialias = kwargs.pop('antialias', '8')
    bw = kwargs.pop('bw', True)
    cam = kwargs.pop('cam', 'iph4s')
    quality = kwargs.pop('quality', 100)
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    sc = bpy.context.scene
    sc.render.use_stamp_lens = True
    sc.render.resolution_percentage = quality
    sc.render.resolution_x = wpx
    sc.render.resolution_y = hpx
    sc.render.antialiasing_samples = antialias
    sc.render.use_overwrite = True
    sc.world.horizon_color = (1, 1, 1)
    if bw:
        sc.render.image_settings.color_mode = 'BW'
    else:
        sc.render.image_settings.color_mode = 'RGB'
    return sc

In [ ]:
def get_camera():
    """Return the first camera ('bpy.types.Camera') of the Blender
    file.('bpy.types.Camera') """
    for i in bpy.data.objects:
        if i.type == 'CAMERA':
            return i
    return -1

In [ ]:
def set_up_cam(cam, idcam='iph4s'):
    """Set up a camera ('bpy.types.Camera') as a real camera.
    
    Add more camera presets to d_cams ('dict') transcribing the values
    from the files in /usr/share/blender/scripts/presets/camera/ to
    the dictionary."""
    d_cams = {'iph4s': ['iPhone 4S', 4.54, 3.42, 4.28,'HORIZONTAL'], }
    cam.data.sensor_width = d_cams[idcam][1]
    cam.data.sensor_height = d_cams[idcam][2]
    cam.data.lens = d_cams[idcam][3]
    cam.data.sensor_fit = d_cams[idcam][4]

In [ ]:
def num_str_zeros(num, n_digs, matlab=False):
    """Return a string that contains a sequence n-zeros followed by
    num ('int') as 'str', for example, num_str_zeros(89, 4) returns
    '0089'. From fwdimaging.ipynbSet a cammera ('bpy.types.Camera')
    as  Jupyter notebook."""
    if matlab:  # Begin the numeration with 1 in the filename.
        num += 1
    str_num = ''
    for i in range(n_digs - len(str(num))):
        str_num += '0'
    str_num += str(num)
    return str_num

In [ ]:
def copy_exif(dest_path, src_path):
    """Copy the Exif metadata of a source image to another."""
    dest_img = Image.open(dest_path)
    wpx = dest_img.size[0]
    hpx = dest_img.size[1]
    dest_img.close()
    src_img = pyexiv2.ImageMetadata(src_path)
    src_img.read()
    dest_img = pyexiv2.ImageMetadata(dest_path)
    dest_img.read()
    src_img.copy(dest_img, exif=True)
    dest_img["Exif.Photo.PixelXDimension"] = wpx
    dest_img["Exif.Photo.PixelYDimension"] = hpx
    dest_img.write()

In [ ]:
def shoot_cam(cam, sc, base_file_path, exif, **kwargs):
    """Render a scene ('bpy.context.scene') with a camera
    ('bpy.types.Camera') and save the result in outpath
    ('str') route as image file."""
    extension = kwargs.pop('extension', 'TIFF')
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    sc.render.image_settings.file_format = extension
    extension = extension.lower()
    if extension == 'jpeg':
        extension = 'jpg'
    sc.camera = cam
    sc.render.filepath = base_file_path
    bpy.ops.render.render(write_still=True)
    file_path = base_file_path + '.' + extension
    copy_exif(file_path, exif)
    print('Saved view with Exif metadata:', file_path)

In [ ]:
def take_photos(cam, sc, l_loc_rot, **kwargs):
    """Take photos with the cam ('bpy.types.Camera') in the
    coordinates and the angles referenced as tuples of l_loc_rot
    ('list')."""    
    s3path = kwargs.pop('s3path',
                        os.path.join(os.path.expanduser('~'),
                                     'super-scanner-software-s3'))
    s3out = kwargs.pop('s3out',
                       os.path.join(os.path.expanduser('~'),
                                    's3-out'))
    dirname = kwargs.pop('dirname', 'blend-phg-set-0001')
    # Path of the directory that will contain the photos to be used
    # with photogrammetry.
    path = kwargs.pop('path', os.path.join(s3out,
                                           'scanner',
                                           dirname))
    prefix = kwargs.pop('prefix', 'view_')
    ext = kwargs.pop('ext', 'TIFF')
    exif = kwargs.pop('exif',
                      os.path.join(s3path,
                                   'img',
                                   'Photo 25-09-16 11 11 00.jpg'))
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(generate_obj.__name__, list(kwargs.keys())[-1]))
    while True: # Create a new folder for the set in the main path.
        if not os.path.exists(path):
            os.makedirs(path)
            break
        else:
            str_num = path.split('-')[-1]
            num_dir = int(str_num)
            num_dir += 1
            str_num_dir = num_str_zeros(num_dir, len(str_num))
            path = '-'.join(path.split('-')[:-1]) + '-' + str_num_dir
    len_llocrot = len(l_loc_rot)
    for i in range(len_llocrot):
        loc, rot = l_loc_rot[i]
        cam.location = loc
        for j in range(3):  # 3 by each rotation.
            cam.rotation_euler[j] = math.radians(rot[j])
        shoot(cam,
              sc,
              os.path.join(path,
                           prefix + num_str_zeros(i,
                                                  len(str(len_llocrot)),
                                                  matlab=True)),
              exif,
              extension=ext)